In [14]:
import pandas as pd
import numpy as np

df = pd.read_csv('level2_i_20221201.csv')
df.head()

long = 100
short = 50


a = df['midp'].diff(long) / df['midp'].shift(long)
a = a.dropna()
a.reset_index(drop = True, inplace = True)
df['long return'] = a

b = df['midp'].diff(short) / df['midp'].shift(short)
b = b.dropna()
b.reset_index(drop = True, inplace = True)
df['short return'] = b

col_reserved = ['fac{}'.format(i) for i in range(1, 34)]
col_reserved.append('long return')
col_reserved.append('short return')

new_df = df[col_reserved]
linear_df = new_df[1200:-1200]
#linear_df.fillna(0, inplace = True)
linear_df.reset_index(drop = True, inplace = True)
linear_df

,fac1,fac2,fac3,fac4,fac5,fac6,fac7,fac8,fac9,fac10,...,fac26,fac27,fac28,fac29,fac30,fac31,fac32,fac33,long return,short return
0,-0.000214,-0.019506,-0.555555,-0.044372,-0.006845,0.012014,0.017698,0.005543,-0.005437,3.0,...,0.0,-0.028571,-0.002507,0.000537,0.000537,0.000493,0.000235,0.000121,0.000647,0.0
1,-0.000292,-0.132730,-0.830189,-0.068396,-0.011363,0.008915,0.016692,0.004849,-0.005456,2.0,...,0.0,-0.048544,-0.002496,0.000523,0.000523,0.000471,0.000226,0.000183,0.000647,0.0
2,-0.000296,-0.017991,-0.999999,-0.067358,-0.010600,0.007809,0.016580,0.004801,-0.005237,0.0,...,0.0,-0.039216,-0.002493,0.000523,0.000523,0.000468,0.000250,0.000201,0.000647,0.0
3,-0.000290,-0.021552,-0.517241,-0.046512,-0.010224,0.007456,0.016282,0.004631,-0.005729,0.0,...,0.0,-0.049505,-0.002485,0.000524,0.000524,0.000468,0.000250,0.000234,0.000647,0.0
4,-0.000322,-0.055639,-1.000000,-0.056485,-0.011385,0.006608,0.015556,0.004362,-0.005393,-1.0,...,0.0,-0.070707,-0.002460,0.000521,0.000521,0.000463,0.000249,0.000224,0.000647,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74345,-0.000054,0.001085,0.333332,0.000816,-0.002180,0.000699,0.004050,-0.004290,-0.006255,0.0,...,-0.0,-0.333333,-0.000835,0.000040,0.000040,0.000171,0.000224,0.000326,0.000000,0.0
74346,-0.000054,-0.001086,-0.999990,0.000815,-0.002074,0.000458,0.004056,-0.004131,-0.006255,0.0,...,-0.0,-0.333333,-0.000835,0.000044,0.000044,0.000171,0.000223,0.000326,0.000000,0.0
74347,-0.000054,-0.001085,-0.999990,0.000271,-0.001803,0.000459,0.004125,-0.003815,-0.006258,0.0,...,-0.0,-0.333333,-0.000835,0.000053,0.000053,0.000171,0.000223,0.000326,0.000000,0.0
74348,-0.000053,0.004329,0.999998,0.000813,0.000382,0.007821,0.004176,-0.002501,-0.006255,1.0,...,0.0,-0.333333,-0.000835,0.000134,0.000134,0.000169,0.000036,0.000109,0.000000,0.0


In [15]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 

window_size = 500

predict_long = []
predict_short = []
# this sequence record the R square of validation set
R_square_long = []
R_square_short = []

#limit = len(linear_df)
limit = 10000

for i in range(window_size, limit):
    train_X = linear_df.iloc[i - window_size : i - 200, :-2]
    train_y_long = linear_df.iloc[i - window_size : i - 200, -2]
    train_y_short = linear_df.iloc[i - window_size : i - 200, -1]
    # split the train set into train set and validation set by order 
    train_X_long, validate_X_long, train_y_long, validate_y_long = train_test_split(train_X, train_y_long, test_size = 0.1, random_state = 11)
    train_X_short, validate_X_short, train_y_short, validate_y_short = train_test_split(train_X, train_y_short, test_size = 0.1, random_state = 11)

    test_X = linear_df.iloc[i:i+1, :-2]
    test_y_long = np.array(linear_df.iloc[i, -2]).reshape(1, -1)
    test_y_short = np.array(linear_df.iloc[i, -1]).reshape(1, -1)

    #model_short = LinearRegression()
    #model_long = LinearRegression()

    model_long = Lasso(alpha = 0.1)
    model_short = Lasso(alpha = 0.1)
    
    scaler_short = StandardScaler()
    scaler_long = StandardScaler()

    scaled_train_X_short = scaler_short.fit_transform(train_X_short)
    scaled_validate_X_short = scaler_short.transform(validate_X_short)
    scaled_test_X_short = scaler_short.transform(test_X)

    scaled_train_X_long = scaler_long.fit_transform(train_X_long)
    scaled_validate_X_long = scaler_long.transform(validate_X_long)
    scaled_test_X_long = scaler_long.transform(test_X)

    model_short.fit(scaled_train_X_short, train_y_short)
    model_long.fit(scaled_train_X_long, train_y_long)

    predict_short.append(model_short.predict(scaled_test_X_short))
    predict_long.append(model_long.predict(scaled_test_X_long))

    R_square_short.append(model_short.score(scaled_validate_X_short, validate_y_short))
    R_square_long.append(model_long.score(scaled_validate_X_long, validate_y_long))

    if i % 1000 == 0:
        print('i = {}'.format(i))
        print('R_square_short = {}'.format(model_short.score(scaled_validate_X_short, validate_y_short)))
        print('R_square_long = {}'.format(model_long.score(scaled_validate_X_long, validate_y_long)))


i = 1000
R_square_short = -0.021124898257484537
R_square_long = -0.004558564444085311
i = 2000
R_square_short = -0.002780730167261325
R_square_long = -0.026504659613841852
i = 3000
R_square_short = -0.0026176814159806217
R_square_long = -0.033327054627172004
i = 4000
R_square_short = -0.00012668570031348914
R_square_long = -0.009613707095719848
i = 5000
R_square_short = -2.298392365318591e-08
R_square_long = -0.014743529921542153


c:\Users\specf\anaconda3\envs\qids-2023-comp\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 0.000e+00, tolerance: 0.000e+00
  model = cd_fast.enet_coordinate_descent(
c:\Users\specf\anaconda3\envs\qids-2023-comp\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 0.000e+00, tolerance: 0.000e+00
  model = cd_fast.enet_coordinate_descent(
c:\Users\specf\anaconda3\envs\qids-2023-comp\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider

i = 6000
R_square_short = -0.008503021487249685
R_square_long = -7.094401321072752e-06
i = 7000
R_square_short = -0.016600948996362552
R_square_long = -0.016730755889752924
i = 8000
R_square_short = -0.026709401709401615
R_square_long = -0.10303307422649044
i = 9000
R_square_short = -0.0011170023131301665
R_square_long = -0.01615884395048628


In [16]:
predict_short = np.array(predict_short).reshape(-1)
predict_long = np.array(predict_long).reshape(-1)
R_square_short = np.array(R_square_short).reshape(-1)
R_square_long = np.array(R_square_long).reshape(-1)

new_df = pd.DataFrame()
new_df['predict_short'] = predict_short
new_df['predict_long'] = predict_long
new_df['R_square_short'] = R_square_short
new_df['R_square_long'] = R_square_long

new_df['long return'] = linear_df['long return'].iloc[window_size:limit].reset_index(drop = True)
new_df['short return'] = linear_df['short return'].iloc[window_size:limit].reset_index(drop = True)

new_df['predict_long_sign'] = new_df['predict_long'].apply(lambda x: 1 if x > 0 else 0 if x == 0 else -1)
new_df['predict_short_sign'] = new_df['predict_short'].apply(lambda x: 1 if x > 0 else 0 if x == 0 else -1)

In [18]:
print(new_df[['predict_short_sign', 'short return']].corr())
print(new_df[['predict_long_sign', 'long return']].corr())
print(new_df[['predict_short', 'short return']].corr())
print(new_df[['predict_long', 'long return']].corr())

                    predict_short_sign  short return
predict_short_sign            1.000000      0.121923
short return                  0.121923      1.000000
                   predict_long_sign  long return
predict_long_sign           1.000000     0.151713
long return                 0.151713     1.000000
               predict_short  short return
predict_short       1.000000      0.089821
short return        0.089821      1.000000
              predict_long  long return
predict_long      1.000000     0.114448
long return       0.114448     1.000000


In [19]:
a = df['midp'].diff(long) / df['midp'].shift(long)
a = a.dropna()
a.reset_index(drop = True, inplace = True)
df['long return'] = a

b = df['midp'].diff(short) / df['midp'].shift(short)
b = b.dropna()
b.reset_index(drop = True, inplace = True)
df['short return'] = b

col_reserved = ['fac{}'.format(i) for i in range(1, 34)]
col_reserved.append('long return')
col_reserved.append('short return')

new_df = df[col_reserved]
linear_df = new_df[1200:-1200]
#linear_df.fillna(0, inplace = True)
linear_df.reset_index(drop = True, inplace = True)

predict_long = []
predict_short = []
# this sequence record the R square of validation set
R_square_long = []
R_square_short = []

for i in range(window_size, limit):
    train_X = linear_df.iloc[i - window_size : i - 200, :-2]
    train_y_long = linear_df.iloc[i - window_size : i - 200, -2]
    train_y_short = linear_df.iloc[i - window_size : i - 200, -1]

    train_X_long, validate_X_long, train_y_long, validate_y_long = train_test_split(train_X, train_y_long, test_size = 0.1, random_state = 11)
    train_X_short, validate_X_short, train_y_short, validate_y_short = train_test_split(train_X, train_y_short, test_size = 0.1, random_state = 11)

    test_X = linear_df.iloc[i:i+1, :-2]
    test_y_long = np.array(linear_df.iloc[i, -2]).reshape(1, -1)
    test_y_short = np.array(linear_df.iloc[i, -1]).reshape(1, -1)

    #model_short = LinearRegression()
    #model_long = LinearRegression()

    model_long = Ridge(alpha = 0.1)
    model_short = Ridge(alpha = 0.1)
    
    scaler_short = StandardScaler()
    scaler_long = StandardScaler()

    scaled_train_X_short = scaler_short.fit_transform(train_X_short)
    scaled_validate_X_short = scaler_short.transform(validate_X_short)
    scaled_test_X_short = scaler_short.transform(test_X)

    scaled_train_X_long = scaler_long.fit_transform(train_X_long)
    scaled_validate_X_long = scaler_long.transform(validate_X_long)
    scaled_test_X_long = scaler_long.transform(test_X)

    model_short.fit(scaled_train_X_short, train_y_short)
    model_long.fit(scaled_train_X_long, train_y_long)

    predict_short.append(model_short.predict(scaled_test_X_short))
    predict_long.append(model_long.predict(scaled_test_X_long))

    R_square_short.append(model_short.score(scaled_validate_X_short, validate_y_short))
    R_square_long.append(model_long.score(scaled_validate_X_long, validate_y_long))

    if i % 1000 == 0:
        print('i = {}'.format(i))
        print('R_square_short = {}'.format(model_short.score(scaled_validate_X_short, validate_y_short)))
        print('R_square_long = {}'.format(model_long.score(scaled_validate_X_long, validate_y_long)))

i = 1000
R_square_short = 0.936636784374368
R_square_long = 0.8843583233664812
i = 2000
R_square_short = 0.6966938926430768
R_square_long = 0.8974168458286829
i = 3000
R_square_short = 0.61595980712705
R_square_long = 0.7485888081074434
i = 4000
R_square_short = 0.844677501323146
R_square_long = 0.9697336926870302
i = 5000
R_square_short = 0.8344372858285898
R_square_long = 0.9690888107999847
i = 6000
R_square_short = 0.8415767414332094
R_square_long = 0.9884390235810417
i = 7000
R_square_short = 0.8272150165587098
R_square_long = 0.8874160951862159
i = 8000
R_square_short = 0.6202930169995982
R_square_long = 0.09600196630833524
i = 9000
R_square_short = 0.856828637134442
R_square_long = 0.9212180507718367


In [20]:
R_square_short = np.array(R_square_short).reshape(-1)
R_square_long = np.array(R_square_long).reshape(-1)

new_df = pd.DataFrame()
new_df['predict_short'] = predict_short
new_df['predict_long'] = predict_long
new_df['R_square_short'] = R_square_short
new_df['R_square_long'] = R_square_long

new_df['long return'] = linear_df['long return'].iloc[window_size:limit].reset_index(drop = True)
new_df['short return'] = linear_df['short return'].iloc[window_size:limit].reset_index(drop = True)
new_df['predict_long_sign'] = new_df['predict_long'].apply(lambda x: 1 if x > 0 else 0 if x == 0 else -1)
new_df['predict_short_sign'] = new_df['predict_short'].apply(lambda x: 1 if x > 0 else 0 if x == 0 else -1)
new_dfpredict_short = np.array(predict_short).reshape(-1)
predict_long = np.array(predict_long).reshape(-1)

In [21]:
print(new_df[['predict_short_sign', 'short return']].corr())
print(new_df[['predict_long_sign', 'long return']].corr())
print(new_df[['predict_short', 'short return']].corr())
print(new_df[['predict_long', 'long return']].corr())

                    predict_short_sign  short return
predict_short_sign            1.000000      0.063666
short return                  0.063666      1.000000
                   predict_long_sign  long return
predict_long_sign           1.000000     0.100886
long return                 0.100886     1.000000
              short return
short return           1.0
             long return
long return          1.0


C:\Users\specf\AppData\Local\Temp\ipykernel_5380\3606610826.py:3: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(new_df[['predict_short', 'short return']].corr())
C:\Users\specf\AppData\Local\Temp\ipykernel_5380\3606610826.py:4: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(new_df[['predict_long', 'long return']].corr())
